## Flight Delay Prediction 
---

Load All the Required Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib as mpl
import seaborn as sns
from datetime import datetime as dt  
from utils.MlUtils import MlUtils ,convert_time ,concat_date_time ,concat_date_time_ARRIVAL, concat_date_time_TaxiOut
from matplotlib.gridspec import GridSpec

In [2]:
ut=MlUtils()
dfFlight = ut.load_csv('././data/flights.csv',  500000)

### `Percentage of missing values per columns`

In [3]:
ut.missingPercent(dfFlight)

                Feature  Missing_Count  Missing_%
0                  YEAR              0     0.0000
1                 MONTH              0     0.0000
2                   DAY              0     0.0000
3           DAY_OF_WEEK              0     0.0000
4               AIRLINE              0     0.0000
5         FLIGHT_NUMBER              0     0.0000
6           TAIL_NUMBER           3750     0.7500
7        ORIGIN_AIRPORT              0     0.0000
8   DESTINATION_AIRPORT              0     0.0000
9   SCHEDULED_DEPARTURE              0     0.0000
10       DEPARTURE_TIME          16416     3.2832
11      DEPARTURE_DELAY          16416     3.2832
12             TAXI_OUT          16704     3.3408
13           WHEELS_OFF          16704     3.3408
14       SCHEDULED_TIME              1     0.0002
15         ELAPSED_TIME          17948     3.5896
16             AIR_TIME          17948     3.5896
17             DISTANCE              0     0.0000
18            WHEELS_ON          17159     3.4318


In [4]:
print(dfFlight.shape)
# Removing because  We have drive the Flight_Delay  from  ARRIVAL_DELAY. So Arrival_Delay is no relevent more.
variables_to_remove = ['FLIGHT_NUMBER', 'TAIL_NUMBER', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY',
                       'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
dfFlight.drop(variables_to_remove, axis=1, inplace=True)
print(dfFlight.shape)


(500000, 31)
(500000, 23)


In [5]:
# dfFlight=dfFlight.iloc[:10,:]
# dfFlight.columns

#### Arriaval Delay is difference between 	`SCHEDULED_ARRIVAL`  and `ARRIVAL_TIME` 

In [6]:
t2=dt.now()
dfFlight['FLIGHT_DELAY'] = dfFlight['ARRIVAL_DELAY'].map( lambda x : 'YES' if x>0 else 'NO')
print(dfFlight.shape)
print(dt.now() -t2)

(500000, 24)
0:00:00.276908


In [7]:
t3=dt.now()
dfFlight['FLIGHT_DATE'] = pd.to_datetime(dfFlight[['YEAR', 'MONTH', 'DAY']])
print(dt.now() -t3)

0:00:00.836532


In [8]:
t4=dt.now()
dfFlight['SCHEDULED_DEPARTURE'] = dfFlight.apply(concat_date_time, axis=1)
print(dt.now() -t4)

0:02:37.837315


In [9]:
t5=dt.now()
dfFlight['DEPARTURE_TIME'] = dfFlight['DEPARTURE_TIME'].map(convert_time)
print(dt.now() -t5)

0:00:52.450534


In [10]:
t6=dt.now()
dfFlight['SCHEDULED_ARRIVAL'] =  dfFlight.apply(concat_date_time_ARRIVAL, axis=1)
print(dfFlight.columns)
print(dfFlight.shape)
print(dt.now() -t6)

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT',
       'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME',
       'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME',
       'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN',
       'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED',
       'CANCELLED', 'FLIGHT_DELAY', 'FLIGHT_DATE'],
      dtype='object')
(500000, 25)
0:01:21.242884


In [12]:
t6=dt.now()
dfFlight['TAXI_OUT'] =  dfFlight.apply(concat_date_time_TaxiOut, axis=1)
print(dfFlight.columns)
print(dfFlight.shape)
print(dt.now() -t6)

ValueError: ("time data '00:61:00' does not match format '%H:%M:%S' (match)", 'occurred at index 502')

### Percentage of missing values per columns

In [ ]:
ut.missingPercent(dfFlight)

### Removing Features which are not more relevant to our model 
___
- `ARRIVAL_DELAY` : Drived `FLIGHT_DELAY`
- `YEAR`, `MONTH`, `DAY` : We drived anohter feature by using them, So i am going to remove them.
- `FLIGHT_NUMBER` : It's flight number like other vehical number. it
- `TAIL_NUMBER` : It's unique identifier for Aircraft. It's called Aircart registration number. 
- `CANCELLATION_REASON` , `AIR_SYSTEM_DELAY` , `SECURITY_DELAY`, `AIRLINE_DELAY`, `LATE_AIRCRAFT_DELAY`, `WEATHER_DELAY` :  Aprox 82% value are missing for these features. 
- `AIR_TIME` : This will be populated after landing. So it's not important feature for our model.
- `FLIGHT_DATE` : Not relevent 
- `ARRIVAL_TIME` ,`WHEELS_ON`, `TAXI_IN` , `CANCELLED` : This will be populated after landing on airport.

- `WHEELS_ON` : The time point that the aircraft's wheels touch on the ground
- `TAXI_IN` : The time duration elapsed between wheels-on and gate arrival at the destination airport
- `ARRIVAL_TIME` : WHEELS_ON+TAXI_IN

- `ELAPSED_TIME` : AIR_TIME+TAXI_IN+TAXI_OUT
 

In [13]:
# Removing because  We have drive the Flight_Delay  from  ARRIVAL_DELAY. So Arrival_Delay is no relevent more.
variables_to_remove=['ARRIVAL_DELAY','YEAR', 'MONTH', 'DAY', 'ELAPSED_TIME' , 'AIR_TIME',
                     'FLIGHT_DATE','ARRIVAL_TIME' , 'WHEELS_ON', 'TAXI_IN','CANCELLED']
# variables_to_remove = [ 'DAY_OF_WEEK',  'TAXI_OUT', 'TAXI_IN', 'WHEELS_ON', 'WHEELS_OFF',  'DIVERTED', 'CANCELLED']
dfFlight.drop(variables_to_remove, axis=1, inplace=True)
print(dfFlight.shape)

(500000, 14)


In [14]:
dfFlight.columns

Index(['DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'DISTANCE', 'SCHEDULED_ARRIVAL',
       'DIVERTED', 'FLIGHT_DELAY'],
      dtype='object')

### Data verification realated to Imbalance or Balance
----

In [15]:
# Calculate Count of classes per class
dfDelayClass=dfFlight['FLIGHT_DELAY'].T.value_counts().reset_index() 
dfDelayClass.columns=["FLIGHT_DELAY_CLASS","COUNT_PER_CLASS"]
dfDelayClass

,FLIGHT_DELAY_CLASS,COUNT_PER_CLASS
0,NO,305778
1,YES,194222


In [16]:
dfDelayClass['%_PER_CLASS']=dfDelayClass['COUNT_PER_CLASS'].map(ut.Missing_Count)
# dMissing_CountfDelayClass
dfDelayClass

,FLIGHT_DELAY_CLASS,COUNT_PER_CLASS,%_PER_CLASS
0,NO,305778,61.1556
1,YES,194222,38.8444


In [17]:
ut.missingPercent(dfFlight=dfFlight)

                Feature  Missing_Count  Missing_%
0           DAY_OF_WEEK              0     0.0000
1               AIRLINE              0     0.0000
2        ORIGIN_AIRPORT              0     0.0000
3   DESTINATION_AIRPORT              0     0.0000
4   SCHEDULED_DEPARTURE              0     0.0000
5        DEPARTURE_TIME          16451     3.2902
6       DEPARTURE_DELAY          16416     3.2832
7              TAXI_OUT          16704     3.3408
8            WHEELS_OFF          16704     3.3408
9        SCHEDULED_TIME              1     0.0002
10             DISTANCE              0     0.0000
11    SCHEDULED_ARRIVAL              0     0.0000
12             DIVERTED              0     0.0000
13         FLIGHT_DELAY              0     0.0000
0:00:00.230739


In [18]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DIVERTED,FLIGHT_DELAY
0,4,AS,ANC,SEA,2015-01-01 00:05:00,23:54:00,-11.0,21.0,15.0,205.0,1448,2015-01-01 04:30:00,0,NO
1,4,AA,LAX,PBI,2015-01-01 00:10:00,00:02:00,-8.0,12.0,14.0,280.0,2330,2015-01-01 07:50:00,0,NO
2,4,US,SFO,CLT,2015-01-01 00:20:00,00:18:00,-2.0,16.0,34.0,286.0,2296,2015-01-01 08:06:00,0,YES
3,4,AA,LAX,MIA,2015-01-01 00:20:00,00:15:00,-5.0,15.0,30.0,285.0,2342,2015-01-01 08:05:00,0,NO
4,4,AS,SEA,ANC,2015-01-01 00:25:00,00:24:00,-1.0,11.0,35.0,235.0,1448,2015-01-01 03:20:00,0,NO


### Airline Details 
---


In [19]:
# Converting Day_of_week in Catogical Feature
week_Days={1:'ONE',2:'TWO',3:'THREE',4:'FOUR',5:'FIVE',6:'SIX',7:'SEVEN'}
delay_binary={'YES':0,'NO':1}

In [20]:
dfFlight['DAY_OF_WEEK']=dfFlight['DAY_OF_WEEK'].map(week_Days)
dfFlight['FLIGHT_DELAY']=dfFlight['FLIGHT_DELAY'].map(delay_binary)
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DIVERTED,FLIGHT_DELAY
0,FOUR,AS,ANC,SEA,2015-01-01 00:05:00,23:54:00,-11.0,21.0,15.0,205.0,1448,2015-01-01 04:30:00,0,1
1,FOUR,AA,LAX,PBI,2015-01-01 00:10:00,00:02:00,-8.0,12.0,14.0,280.0,2330,2015-01-01 07:50:00,0,1
2,FOUR,US,SFO,CLT,2015-01-01 00:20:00,00:18:00,-2.0,16.0,34.0,286.0,2296,2015-01-01 08:06:00,0,0
3,FOUR,AA,LAX,MIA,2015-01-01 00:20:00,00:15:00,-5.0,15.0,30.0,285.0,2342,2015-01-01 08:05:00,0,1
4,FOUR,AS,SEA,ANC,2015-01-01 00:25:00,00:24:00,-1.0,11.0,35.0,235.0,1448,2015-01-01 03:20:00,0,1


#### Feature Engineering 
---
`We will convert all the time in minutes in Decimal`

- `ORIGIN_AIRPORT` `DESTINATION_AIRPORT` : `LAG_INFO`


In [21]:
dfFlight['LAG_INFO']= dfFlight['ORIGIN_AIRPORT']+ '--' +dfFlight['DESTINATION_AIRPORT']
dfFlight.shape

(500000, 15)

In [22]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DIVERTED,FLIGHT_DELAY,LAG_INFO
0,FOUR,AS,ANC,SEA,2015-01-01 00:05:00,23:54:00,-11.0,21.0,15.0,205.0,1448,2015-01-01 04:30:00,0,1,ANC--SEA
1,FOUR,AA,LAX,PBI,2015-01-01 00:10:00,00:02:00,-8.0,12.0,14.0,280.0,2330,2015-01-01 07:50:00,0,1,LAX--PBI
2,FOUR,US,SFO,CLT,2015-01-01 00:20:00,00:18:00,-2.0,16.0,34.0,286.0,2296,2015-01-01 08:06:00,0,0,SFO--CLT
3,FOUR,AA,LAX,MIA,2015-01-01 00:20:00,00:15:00,-5.0,15.0,30.0,285.0,2342,2015-01-01 08:05:00,0,1,LAX--MIA
4,FOUR,AS,SEA,ANC,2015-01-01 00:25:00,00:24:00,-1.0,11.0,35.0,235.0,1448,2015-01-01 03:20:00,0,1,SEA--ANC


In [23]:
# Removing because  We have drive the Flight_Delay  from  ARRIVAL_DELAY. So Arrival_Delay is no relevent more.
variables_to_remove=['DESTINATION_AIRPORT','ORIGIN_AIRPORT']
dfFlight.drop(variables_to_remove, axis=1, inplace=True)
print(dfFlight.shape)

(500000, 13)


In [24]:
print(dfFlight['LAG_INFO'].nunique())
dfFlight.head()

4180


,DAY_OF_WEEK,AIRLINE,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DIVERTED,FLIGHT_DELAY,LAG_INFO
0,FOUR,AS,2015-01-01 00:05:00,23:54:00,-11.0,21.0,15.0,205.0,1448,2015-01-01 04:30:00,0,1,ANC--SEA
1,FOUR,AA,2015-01-01 00:10:00,00:02:00,-8.0,12.0,14.0,280.0,2330,2015-01-01 07:50:00,0,1,LAX--PBI
2,FOUR,US,2015-01-01 00:20:00,00:18:00,-2.0,16.0,34.0,286.0,2296,2015-01-01 08:06:00,0,0,SFO--CLT
3,FOUR,AA,2015-01-01 00:20:00,00:15:00,-5.0,15.0,30.0,285.0,2342,2015-01-01 08:05:00,0,1,LAX--MIA
4,FOUR,AS,2015-01-01 00:25:00,00:24:00,-1.0,11.0,35.0,235.0,1448,2015-01-01 03:20:00,0,1,SEA--ANC


- departure timeing AM/PM
- `SCHEDULED_DEPARTURE` : Planned Departure Time
- `DEPARTURE_TIME` : WHEEL_OFF - TAXI_OUT
- `DEPARTURE_DELAY` : Total Delay on Departure
- `TAXI_OUT` : The time duration elapsed between departure from the origin airport gate and wheels off
- `WHEELS_OFF` : The time point that the aircraft's wheels leave the ground
- `SCHEDULED_TIME` : Planned time amount needed for the flight trip
- `AIR_TIME` : The time duration between wheels_off and wheels_on time
- `DISTANCE` : Distance between two airports

- `SCHEDULED_ARRIVAL` : Planned arrival time

- `ARRIVAL_DELAY` : ARRIVAL_TIME-SCHEDULED_ARRIVAL
- `DIVERTED` : Aircraft landed on airport that out of schedule

In [25]:
# convert time to decimal minutes

#     rawtime = pd.to_datetime(t, format='%H:%M:%S').time() 
#     print(t.hour, rawtime.minute)
#      m = (hours * 60) + minutes + (seconds / 60). Ti
#     times_in_decimal=(t.hour * 3600) + (t.minute * 60) 
def convert_hour_to_decimal_minute(rawtime):
    if pd.isna(rawtime) :
        np.nan
    else:
#         print(type(rawtime),rawtime)
        times_in_decimal = (rawtime.hour * 60) + rawtime.minute 
        #     print(times_in_decimal)
        return times_in_decimal

In [26]:
# dfTest=dfFlight.loc[:5,:]
dfFlight['DEPARTURE_TIME_DECIMAL']=dfFlight['DEPARTURE_TIME'].map(convert_hour_to_decimal_minute)
dfFlight.shape

(500000, 14)

In [27]:
# dfFlight = dfFlight.reindex(sorted(dfFlight.columns), axis=1)
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DIVERTED,FLIGHT_DELAY,LAG_INFO,DEPARTURE_TIME_DECIMAL
0,FOUR,AS,2015-01-01 00:05:00,23:54:00,-11.0,21.0,15.0,205.0,1448,2015-01-01 04:30:00,0,1,ANC--SEA,1434.0
1,FOUR,AA,2015-01-01 00:10:00,00:02:00,-8.0,12.0,14.0,280.0,2330,2015-01-01 07:50:00,0,1,LAX--PBI,2.0
2,FOUR,US,2015-01-01 00:20:00,00:18:00,-2.0,16.0,34.0,286.0,2296,2015-01-01 08:06:00,0,0,SFO--CLT,18.0
3,FOUR,AA,2015-01-01 00:20:00,00:15:00,-5.0,15.0,30.0,285.0,2342,2015-01-01 08:05:00,0,1,LAX--MIA,15.0
4,FOUR,AS,2015-01-01 00:25:00,00:24:00,-1.0,11.0,35.0,235.0,1448,2015-01-01 03:20:00,0,1,SEA--ANC,24.0


In [28]:
dfFlight.dtypes

DAY_OF_WEEK                       object
AIRLINE                           object
SCHEDULED_DEPARTURE       datetime64[ns]
DEPARTURE_TIME                    object
DEPARTURE_DELAY                  float64
TAXI_OUT                         float64
WHEELS_OFF                       float64
SCHEDULED_TIME                   float64
DISTANCE                           int64
SCHEDULED_ARRIVAL         datetime64[ns]
DIVERTED                           int64
FLIGHT_DELAY                       int64
LAG_INFO                          object
DEPARTURE_TIME_DECIMAL           float64
dtype: object

In [29]:
dfFlight['SCHEDULED_ARRIVAL_DECIMAL']=dfFlight['SCHEDULED_ARRIVAL'].map(convert_hour_to_decimal_minute)

In [30]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DIVERTED,FLIGHT_DELAY,LAG_INFO,DEPARTURE_TIME_DECIMAL,SCHEDULED_ARRIVAL_DECIMAL
0,FOUR,AS,2015-01-01 00:05:00,23:54:00,-11.0,21.0,15.0,205.0,1448,2015-01-01 04:30:00,0,1,ANC--SEA,1434.0,270
1,FOUR,AA,2015-01-01 00:10:00,00:02:00,-8.0,12.0,14.0,280.0,2330,2015-01-01 07:50:00,0,1,LAX--PBI,2.0,470
2,FOUR,US,2015-01-01 00:20:00,00:18:00,-2.0,16.0,34.0,286.0,2296,2015-01-01 08:06:00,0,0,SFO--CLT,18.0,486
3,FOUR,AA,2015-01-01 00:20:00,00:15:00,-5.0,15.0,30.0,285.0,2342,2015-01-01 08:05:00,0,1,LAX--MIA,15.0,485
4,FOUR,AS,2015-01-01 00:25:00,00:24:00,-1.0,11.0,35.0,235.0,1448,2015-01-01 03:20:00,0,1,SEA--ANC,24.0,200


In [31]:
df=pd.get_dummies(dfFlight, drop_first=False)
df.columns

Index(['SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'SCHEDULED_TIME', 'DISTANCE', 'SCHEDULED_ARRIVAL', 'DIVERTED',
       'FLIGHT_DELAY', 'DEPARTURE_TIME_DECIMAL',
       ...
       'LAG_INFO_XNA--DEN', 'LAG_INFO_XNA--DFW', 'LAG_INFO_XNA--EWR',
       'LAG_INFO_XNA--IAH', 'LAG_INFO_XNA--LGA', 'LAG_INFO_XNA--MSP',
       'LAG_INFO_XNA--ORD', 'LAG_INFO_YAK--CDV', 'LAG_INFO_YAK--JNU',
       'LAG_INFO_YUM--PHX'],
      dtype='object', length=5617)

In [32]:
df.shape

(500000, 5617)

In [33]:
dfAirline=ut.load_csv('././data/airlines.csv',numRow=300)

In [ ]:
airline_companies = dfAirline.set_index('IATA_CODE')['AIRLINE'].to_dict()

In [ ]:
airline_companies

In [ ]:
# dfFlight(['FLIGHT_DELAY','AIRLINE']).groupby(['AIRLINE']).agg('count')
dfGroup=dfFlight.groupby(['AIRLINE','FLIGHT_DELAY'])

In [ ]:
dfGroup=dfGroup['FLIGHT_DELAY'].agg(['count']).reset_index()

In [ ]:
dfGroup

In [ ]:
dfGroup
dfGroup.columns=['AIRLINE','FLIGHT_DELAY','DELAY_COUNT']

In [ ]:
dfGroup['DELAY_COUNT_%']=dfGroup['DELAY_COUNT'].map(missing_percent)

#### Delay Analysis in respect to Air line 

In [ ]:
dfGroup['AIRLINE']=dfGroup['AIRLINE'].map(airline_companies)

In [ ]:
dfGroup

In [ ]:
# function that extract statistical parameters from a grouby objet:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
#_______________________________________________________________
# Creation of a dataframe with statitical infos on each airline:
global_stats = dfFlight['DEPARTURE_DELAY'].groupby(dfFlight['AIRLINE']).apply(get_stats).unstack()
global_stats = global_stats.sort_values('count')
global_stats

In [ ]:
font = {'family' : 'normal', 'weight' : 'bold', 'size'   : 15}
mpl.rc('font', **font)

In [ ]:
#__________________________________________________________________
# I extract a subset of columns and redefine the airlines labeling 
df2 = dfFlight.loc[:, ['AIRLINE', 'DEPARTURE_DELAY']]
df2['AIRLINE'] = df2['AIRLINE'].replace(airline_companies)
#________________________________________________________________________
colors = ['royalblue', 'grey', 'wheat', 'c', 'firebrick', 'seagreen', 'lightskyblue',
          'lightcoral', 'yellowgreen', 'gold', 'tomato', 'violet', 'aquamarine', 'chartreuse']
#___________________________________
fig = plt.figure(1, figsize=(16,15))
gs=GridSpec(2,2)             
ax1=fig.add_subplot(gs[0,0]) 
ax2=fig.add_subplot(gs[0,1]) 
ax3=fig.add_subplot(gs[1,:]) 
#------------------------------
# Pie chart nº1: nb of flights
#------------------------------
labels = [s for s in  global_stats.index]
sizes  = global_stats['count'].values
explode = [0.3 if sizes[i] < 20000 else 0.0 for i in range(len(airline_companies))]
patches, texts, autotexts = ax1.pie(sizes, explode = explode,
                                labels=labels, colors = colors,  autopct='%1.0f%%',
                                shadow=False, startangle=0)
for i in range(len(airline_companies)): 
    texts[i].set_fontsize(14)
ax1.axis('equal')
ax1.set_title('% of flights per company', bbox={'facecolor':'midnightblue', 'pad':5},
              color = 'w',fontsize=18)
#_______________________________________________
# I set the legend: abreviation -> airline name
comp_handler = []
for i in range(len(airline_companies)):
    comp_handler.append(mpatches.Patch(color=colors[i],
            label = global_stats.index[i] + ': ' + airline_companies[global_stats.index[i]]))
ax1.legend(handles=comp_handler, bbox_to_anchor=(0.2, 0.9), 
           fontsize = 13, bbox_transform=plt.gcf().transFigure)
#----------------------------------------
# Pie chart nº2: mean delay at departure
#----------------------------------------
sizes  = global_stats['mean'].values
sizes  = [max(s,0) for s in sizes]
explode = [0.0 if sizes[i] < 20000 else 0.01 for i in range(len(airline_companies))]
patches, texts, autotexts = ax2.pie(sizes, explode = explode, labels = labels,
                                colors = colors, shadow=False, startangle=0,
                                autopct = lambda p :  '{:.0f}'.format(p * sum(sizes) / 100))
for i in range(len(airline_companies)): 
    texts[i].set_fontsize(14)
ax2.axis('equal')
ax2.set_title('Mean delay at origin', bbox={'facecolor':'midnightblue', 'pad':5},
              color='w', fontsize=18)
#------------------------------------------------------
# striplot with all the values reported for the delays
#___________________________________________________________________
# I redefine the colors for correspondance with the pie charts
colors = ['firebrick', 'gold', 'lightcoral', 'aquamarine', 'c', 'yellowgreen', 'grey',
          'seagreen', 'tomato', 'violet', 'wheat', 'chartreuse', 'lightskyblue', 'royalblue']
#___________________________________________________________________
ax3 = sns.stripplot(y="AIRLINE", x="DEPARTURE_DELAY", size = 4, palette = colors,
                    data=df2, linewidth = 0.5,  jitter=True)
plt.setp(ax3.get_xticklabels(), fontsize=14)
plt.setp(ax3.get_yticklabels(), fontsize=14)
ax3.set_xticklabels(['{:2.0f}h{:2.0f}m'.format(*[int(y) for y in divmod(x,60)])
                         for x in ax3.get_xticks()])
plt.xlabel('Departure delay', fontsize=18, bbox={'facecolor':'midnightblue', 'pad':5},
           color='w', labelpad=20)
ax3.yaxis.label.set_visible(False)
#________________________
plt.tight_layout(w_pad=3) 